# 1b. Export RT-IGEV++ to ONNX

Converts an **RT-IGEV++** (real-time variant) PyTorch checkpoint to an
ONNX model that can be loaded by
`ag-cam-tools depth-preview --stereo-backend onnx`.

RT-IGEV++ is significantly faster than the full IGEV++ model while
maintaining strong accuracy. Key simplifications:

- Single shared backbone (no separate context network)
- Single-range geometry encoding volume (vs. multi-range)
- Single-level ConvGRU with 96 hidden channels (vs. 3-level with 128)
- Default `max_disp=192`, `iters=8`

The exported model expects two float32 inputs of shape `[1, 3, H, W]` in
`[0, 255]` range and produces a float32 disparity map.

**This export is tied to a specific resolution and max_disp.** If you
recalibrate or change binning, re-run this notebook.

---

### Prerequisites

1. Clone the IGEV++ repository:
   ```bash
   git clone https://github.com/gangweiX/IGEV-plusplus.git ~/code/IGEV-plusplus
   ```

2. Download the RT-IGEV++ SceneFlow checkpoint from the
   [Google Drive folder](https://drive.google.com/drive/folders/1eubNsu03MlhUfTtrbtN7bfAsl39s2ywJ):
   ```bash
   mkdir -p ~/code/IGEV-plusplus/pretrained_models/rt_igev_plusplus
   # Download sceneflow.pth into that directory.
   ```

3. Have a completed calibration session (from `ag-cam-tools calibration-capture`
   + `2.Calibration.ipynb`).

### Setup (one-time)
```bash
cd backends
python3 -m venv .venv
source .venv/bin/activate
pip install -r requirements-onnx-export.txt
python -m ipykernel install --user --name agrippa-export --display-name "agrippa-export"
```

Then select the **agrippa-export** kernel in Jupyter before running.

## Dependencies

In [ ]:
%pip install -r requirements-onnx-export.txt -q

In [ ]:
import contextlib
import json
import os
import struct
import sys
import time

import numpy as np
import torch

## Configuration

Set these paths to match your setup.  `calibration_session` should point
to a calibrated session folder containing `calib_result/calibration_meta.json`
and `calib_result/remap_left.bin`.

In [ ]:
# Path to cloned IGEV-plusplus repo root (contains core_rt/).
igev_src = os.path.expanduser("~/code/IGEV-plusplus")

# Path to pre-trained RT-IGEV++ checkpoint.
checkpoint_path = os.path.join(
    igev_src, "pretrained_models/rt_igev_plusplus/sceneflow.pth")

# Calibration session folder.
calibration_session = "../calibration/calibration_20260225_160029_3cb907d2"

# GRU refinement iterations (8 is the upstream default for RT-IGEV++).
iters = 8

# Output ONNX file path.
output_path = "../models/rt_igev_plusplus.onnx"

# Optional overrides (set to None to auto-detect from calibration).
override_max_disp = None   # e.g. 192
override_width    = None   # e.g. 720
override_height   = None   # e.g. 540

## Load calibration metadata

Reads `calibration_meta.json` to derive `max_disp` (cost volume size)
and the remap table header to get the image dimensions.

In [ ]:
# --- Read max_disp from calibration_meta.json ---
meta_path = os.path.join(calibration_session, "calib_result",
                         "calibration_meta.json")
with open(meta_path, "r") as f:
    meta = json.load(f)

dr = meta.get("disparity_range", {})
num_disparities = dr.get("num_disparities", 128)

# RT-IGEV++ uses a single-range cost volume with D = max_disp // 4.
# The 3D hourglass has 3 stride-2 down/up stages, so D must be
# divisible by 8.  This means max_disp must be divisible by 32.
#
# The upstream default is 192 (D=48, 48/8=6).
# We round up to the next multiple of 32 that covers num_disparities.
raw = override_max_disp or max(num_disparities, 192)
max_disp = ((raw + 31) // 32) * 32

print(f"max_disp = {max_disp}  (calibration num_disparities = {num_disparities})")

# --- Read image dimensions from remap table header ---
# Binary format: "RMAP" (4 bytes) + width(u32) + height(u32) + flags(u32)
width = override_width
height = override_height

if width is None or height is None:
    remap_path = os.path.join(calibration_session, "calib_result",
                              "remap_left.bin")
    with open(remap_path, "rb") as f:
        magic = f.read(4)
        assert magic == b"RMAP", f"Bad remap magic: {magic!r}"
        w, h, _flags = struct.unpack("<III", f.read(12))
        if width is None:
            width = w
        if height is None:
            height = h

def pad_to_32(dim):
    """Round up to nearest multiple of 32."""
    return ((dim + 31) // 32) * 32

pad_w = pad_to_32(width)
pad_h = pad_to_32(height)

print(f"Frame dimensions: {width}x{height}")
print(f"Padded (32-divisible): {pad_w}x{pad_h}")

## Patch autocast for CPU tracing

RT-IGEV++ uses `torch.cuda.amp.autocast` in its forward pass and in
`core_rt/update.py`.  We replace it with a no-op context manager so
ONNX tracing works on CPU-only machines.

In [ ]:
class _NoOpAutocast:
    """Drop-in replacement for torch.cuda.amp.autocast."""
    def __init__(self, *args, **kwargs):
        pass
    def __enter__(self):
        return self
    def __exit__(self, *args):
        pass
    def __call__(self, func):
        return func

if hasattr(torch.cuda, "amp"):
    torch.cuda.amp.autocast = _NoOpAutocast

if hasattr(torch, "autocast"):
    _orig_autocast = torch.autocast
    def _patched_autocast(device_type="cuda", *args, **kwargs):
        if device_type == "cuda":
            return contextlib.nullcontext()
        return _orig_autocast(device_type, *args, **kwargs)
    torch.autocast = _patched_autocast

print("Patched autocast for CPU tracing.")

## Load RT-IGEV++ model

Imports the RT-IGEV++ model from `core_rt/`, builds the configuration
args it expects, and loads the pre-trained checkpoint.

In [ ]:
# Add IGEV++ source to path.
sys.path.insert(0, igev_src)

from core_rt.rt_igev_stereo import IGEVStereo
print("Imported IGEVStereo from core_rt.rt_igev_stereo")

# Build args namespace that RT-IGEV++ expects.
# Much simpler than full IGEV++ — single-range volume, scalar hidden_dim.
class ModelArgs:
    pass

args = ModelArgs()
args.max_disp = max_disp
args.valid_iters = iters
args.mixed_precision = False
args.precision_dtype = "float32"
args.corr_levels = 2
args.corr_radius = 4
args.n_downsample = 2
args.n_gru_layers = 3
args.hidden_dim = 96

model = IGEVStereo(args)
print(f"Model instantiated (max_disp={max_disp}, hidden_dim=96, iters={iters})")

# Load checkpoint.
# weights_only=False is required — the upstream checkpoint was saved with
# older PyTorch and contains objects that the safe unpickler rejects.
print(f"Loading checkpoint: {checkpoint_path}")
state_dict = torch.load(checkpoint_path, map_location="cpu", weights_only=False)

# Handle DataParallel wrapper prefix.
if any(k.startswith("module.") for k in state_dict.keys()):
    state_dict = {k.replace("module.", ""): v
                  for k, v in state_dict.items()}
    print("Stripped 'module.' prefix from state_dict.")

model.load_state_dict(state_dict, strict=False)
model.eval()
print("Checkpoint loaded.")

## Export to ONNX

Traces the model at the padded resolution with `torch.onnx.export` at
opset 16.  The dummy inputs use `[0, 255]` float32 range (not `[0, 1]`)
which matches what the C backend will send.

In [ ]:
# Ensure output directory exists.
out_dir = os.path.dirname(output_path)
if out_dir:
    os.makedirs(out_dir, exist_ok=True)

# RT-IGEV++ expects [0, 255] float32 input.
left_dummy = torch.randn(1, 3, pad_h, pad_w) * 128.0 + 128.0
right_dummy = torch.randn(1, 3, pad_h, pad_w) * 128.0 + 128.0

# Set test_mode to get single output (final refinement only).
model.test_mode = True

print(f"Tracing with opset 16 at {pad_w}x{pad_h} (iters={iters})...")
t0 = time.time()

with torch.no_grad():
    torch.onnx.export(
        model,
        (left_dummy, right_dummy),
        output_path,
        opset_version=16,
        input_names=["left", "right"],
        output_names=["disparity"],
        dynamic_axes=None,  # fixed resolution for best performance
        do_constant_folding=True,
        dynamo=False,  # force legacy JIT-trace exporter (dynamo can't handle IGEV++)
    )

dt = time.time() - t0
size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f"Raw ONNX exported in {dt:.1f}s ({size_mb:.1f} MB)")

## Simplify with onnxsim

Runs graph optimizations (constant folding, shape inference, dead-node
elimination) to reduce model size and improve inference speed.

In [ ]:
import onnx
from onnxsim import simplify

print("Simplifying with onnxsim...")
t0 = time.time()

onnx_model = onnx.load(output_path)
model_sim, check = simplify(onnx_model)

if check:
    onnx.save(model_sim, output_path)
    dt = time.time() - t0
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"Simplified in {dt:.1f}s ({size_mb:.1f} MB)")
else:
    print("onnxsim check failed, keeping unsimplified model.")

## Validate with ONNX Runtime

Loads the exported model in ONNX Runtime and runs a single inference pass
to verify the model is valid and produces reasonable output.

In [ ]:
import onnxruntime as ort

session = ort.InferenceSession(output_path,
                               providers=["CPUExecutionProvider"])

inputs = session.get_inputs()
outputs = session.get_outputs()
print(f"Inputs:  {[(i.name, i.shape, i.type) for i in inputs]}")
print(f"Outputs: {[(o.name, o.shape, o.type) for o in outputs]}")

# Warm-up / validation inference.
left = np.random.uniform(0, 255, (1, 3, pad_h, pad_w)).astype(np.float32)
right = np.random.uniform(0, 255, (1, 3, pad_h, pad_w)).astype(np.float32)

t0 = time.time()
results = session.run(None, {inputs[0].name: left, inputs[1].name: right})
dt = time.time() - t0

disp = results[-1].squeeze()
print(f"\nInference time: {dt:.2f}s")
print(f"Output shape: {disp.shape}")
print(f"Disparity range: [{disp.min():.2f}, {disp.max():.2f}]")

if disp.shape != (pad_h, pad_w):
    print(f"\nWARNING: output shape {disp.shape} != expected ({pad_h}, {pad_w})")
else:
    print("\nValidation OK.")

## Done

The RT-IGEV++ ONNX model has been exported, simplified, and validated.

### Next steps

1. Copy the model to your target machine:
   ```bash
   scp models/rt_igev_plusplus.onnx jetson:~/agrippa-stereocam/models/
   ```

2. Run depth preview:
   ```bash
   ag-cam-tools depth-preview \
       --rectify calibration/calibration_YYYYMMDD_HHMMSS \
       --stereo-backend onnx \
       --model-path models/rt_igev_plusplus.onnx
   ```

The C backend automatically selects the best execution provider
(CUDA > CPU) and handles padding/preprocessing.